In [ ]:
!pip install objectnat
# Installing the 'objectnat' library which provides functions for working with multimodal graphs and adjacency matrices

In [ ]:
!pip install pyarrow 
# Installing 'pyarrow' to read and write .parquet files, a columnar storage file format.

In [1]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

# Importing necessary libraries:
# - 'objectnat' provides the graph and adjacency matrix calculation tools
# - 'geopandas' for handling geospatial data
# - 'pandas' for working with tabular data
# - 'shapely.ops.unary_union' to combine geometries

buildings = gpd.read_parquet('examples_data/buildings.parquet')
services = gpd.read_parquet('examples_data/services.parquet')
# Reading the building and service data from .parquet files into GeoDataFrames

polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001)
# Creating a union of all building and service geometries.
# Then generating a convex hull around these geometries and buffering it slightly (0.001 units) 
# to create an encircling polygon that encompasses the data for later use.

G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True)
# Using the encircling polygon to download and clip an intermodal transportation network graph
# 'get_intermodal_graph' fetches a multimodal graph (e.g., walking, public transport) within the given polygon bounds
# you can also use your custom polygon or osm_id of the territory

buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True)
# Reprojecting the 'buildings' and 'services' GeoDataFrames to match the coordinate reference system (CRS) of the graph

2024-10-31 12:02:28.794 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:214 - Downloading walk graph from OSM, it may take a while for large territory ...


Parsing public transport routes:   0%|          | 0/53 [00:00<?, ?it/s]

Calculating the weights of the walk graph:   0%|          | 0/46614 [00:00<?, ?it/s]

2024-10-31 12:02:39.308 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:50 - Composing intermodal graph...


In [6]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gdf_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 threshold=45
                                                 )
# Creating an adjacency matrix that shows the shortest travel time in minutes ('time_min')
# between each building and service, based on the intermodal graph.
# 'get_adj_matrix_gdf_to_gdf' takes the buildings (gdf_from), services (gdf_to),
# and the graph (nx_graph) and calculates travel times as edge weights.

In [ ]:
matrix.to_parquet('examples_data/matrix_time.parquet')
# Saving the resulting adjacency matrix to a .parquet file for future use.